In [ ]:
import os
import workflow_utils_v3
import sys

from workflow_utils_v3.FileDirectory import Directory

dirs = Directory(rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2024/')

# Sets directory of entire package
# rootpath = '/data/tigusa1/MLTO_UCAH/MLTO_2023/'

nbpath = os.path.join(dirs._8_CNN3D_multiparam_regressor, 'CNN3D_paper_iterations/multi_MLP')
cp_dir = os.path.join(nbpath, 'model_CPs')
# params = ['CH_11 scaled', 'CH_22 scaled', 'CH_44 scaled', 'EH_11 scaled', 'vH_12 scaled', 'KH_11 scaled', 'kappaH_11 scaled'] # old


from torchsummary import summary

import pandas as pd
import numpy as np
import json
import glob
import os

# For plotting
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from itertools import cycle
from plotly.colors import sequential, qualitative

from workflow_utils_v3.Dataset_Preprocessor import Dataset_Preprocessor as DataP
source_data_path = dirs.source_data_path

from workflow_utils.DatasetCreator import DatasetCreator as DC
source_data_path = dirs.source_data_path

import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F


import torch.utils.data as data
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch.optim as optim
# import torchsummary

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from workflow_utils.CNN3D_Compressed_Spatial import ResidualBlock
from workflow_utils.Training_Utils import VoxelDataset, train, validate

In [ ]:
# This dataset creator class modified for Regression VAE

class Dynamic_Dataset(data.Dataset):

    def __init__(self, Strain, Stress, Feature_Map_Vector, UnitCell_Parameters: List):

        super().__init__()
        self.strain = Strain
        self.X_feats = Feature_Map_Vector
        self.Xparams= UnitCell_Parameters
        self.Y_stress = Stress
        self.size = Stress.shape[0]
        self.setup_dataset(self.Xarray, self.Yarray)

    # def generate_continuous_xor(self):
    def setup_dataset(self, Xarray, Yarray):
        data = Xarray
        paramvec = Yarray

        self.data = data
        self.paramvec = paramvec

    def __len__(self):
        # Number of data point we have. Alternatively self.data.shape[0], or self.label.shape[0]
        return self.size

    def __getitem__(self, idx):
        



In [ ]:

def PINN_loss(sigma_pred, sigma_actual, strain, A, B, C):
    

    data_loss = nn.L1Loss()(sigma_pred, sigma_actual)
    
    phys_loss = A*torch.log(strain) + B*(C**(B/C))
    
    total_loss =  phys_loss + data_loss    
    
    return total_loss, phys_loss, data_loss
